In [6]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [7]:
import numpy as np
import pandas
import seaborn
import matplotlib.pyplot as plt

In [8]:
rng = np.random.default_rng()

# Basic bootstrapping code

In [9]:
def get_n_samples_from_empirical_distribution_of_base_sample(base_sample, n):
    samples = []
    for i in range(n):
        samples.append(rng.choice(base_sample))
    return samples

def get_one_sample_from_boostrap_estimate_of_estimator_distribution(base_sample, estimation_function):
    resample = get_n_samples_from_empirical_distribution_of_base_sample(base_sample, len(base_sample))
    return estimation_function(resample)

def get_n_samples_from_boostrap_estimate_of_estimator_distribution(base_sample, estimation_function, n):
    samples = []
    for i in range(n):
        samples.append(get_one_sample_from_boostrap_estimate_of_estimator_distribution(base_sample, estimation_function))
    return samples

In [10]:
# Simple test
base_sample = np.random.randn(400)
estimation_function = np.mean
samples = get_n_samples_from_boostrap_estimate_of_estimator_distribution(base_sample, estimation_function, n=1000)
seaborn.displot(data=samples, kind='hist', stat='density')
print(np.array(samples).std())
print(np.mean(base_sample))
print(np.mean(samples))

0.051611577445550214
0.01704420668792893
0.020207057703826917


# Risk graph

In [4]:
x1 = np.linspace(440, 500, 100)
y1 = 10**((x1-10)/20-25)
x2 = np.linspace(350, 440, 100)
y2 = 10**((x2-10)/5-89.5)
x = np.concatenate([x1, x2])
y = np.concatenate([y1, y2])
seaborn.relplot(x=x, y=y, kind='line', aspect=1.5, linewidth=5)
plt.yscale('log')
plt.xlabel('K', fontsize=18)
plt.ylabel('Probabilité de fusion du cœur\n sur la durée vie de la centrale', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks([1, 1e-03, 1e-06, 1e-09, 1e-12, 1e-15, 1e-18], fontsize=18)
plt.grid()
plt.tight_layout()
plt.savefig('fusion_proba.pdf')

# Small samples

In [5]:
rounded_F = [11.5, 0.02, 779, 13.9, 1124, 526, 0.0005, 594]
rounded_N = [488, 631, 110, 432, 444, 705, 178]
print(np.mean(rounded_F))
print(np.mean(rounded_N))

381.0525625
426.85714285714283


In [ ]:
seaborn.displot(rounded_F, kind='hist', stat='density', bins=[0, 250, 500, 750, 1000, 1250])
plt.xlabel('K', fontsize=18)
plt.ylabel('Densité', fontsize=18)
plt.tight_layout()
plt.savefig('hat_PF.pdf')

In [ ]:
def sample_PF():
    sample = []
    for i in range(8):
        flip = rng.integers(2)
        if flip == 1:
            sample.append(379)
        else:
            sample.append(383)
    return sample

means = []
for i in range(40):
    sample = sample_PF()
    means.append(np.mean(sample))

In [ ]:
values, counts = np.unique(means, return_counts=True)
seaborn.catplot(x=values, y=counts/40, kind='bar')
plt.ylabel('Masse de probabilité estimée', fontsize=18)
plt.xlabel(r'$\hat{K}_F$', fontsize=18)
plt.tight_layout()
plt.savefig('distrib_for_known_PF.pdf')

In [ ]:
bF = get_n_samples_from_boostrap_estimate_of_estimator_distribution(rounded_F, np.mean, n=40)
#bN = get_n_samples_from_boostrap_estimate_of_estimator_distribution(rounded_N, np.mean, n=1000)
dbF = pandas.DataFrame({r'$\hat{K}_F$': bF})
#dbN = pandas.DataFrame({'mean_fusion_proba': bN, 'technology': ['N']*len(bN)})
seaborn.displot(data=dbF, x=r'$\hat{K}_F$', kind='hist', stat='density')
plt.ylabel('Densité', fontsize=18)
plt.xlabel(r'$\hat{K}_F$', fontsize=18)
plt.tight_layout()
plt.savefig('boostrap_distribution_hat_KF.pdf')

### below is how I drew the samples above initially

In [ ]:
D_F = rng.gamma(.16, 2500, 8)
print(D_F)

In [ ]:
D_N = rng.gamma(4.1, 100, 8)
print(D_N)

In [ ]:
seaborn.displot(D_N)
print(np.mean(D_N))

In [ ]:
seaborn.displot(D_F)
print(np.mean(D_F))

In [ ]:
bF = get_n_samples_from_boostrap_estimate_of_estimator_distribution(D_F, np.mean, n=1000)
bN = get_n_samples_from_boostrap_estimate_of_estimator_distribution(D_N, np.mean, n=1000)
dbF = pandas.DataFrame({'mean_fusion_proba': bF, 'technology': ['F']*len(bF)})
dbN = pandas.DataFrame({'mean_fusion_proba': bN, 'technology': ['N']*len(bN)})
b = pandas.concat([dbF, dbN])
seaborn.displot(data=b, x='mean_fusion_proba', hue='technology', kind='hist', stat='density', common_norm=False)

# Analysis with 5000 samples and 1000 resamples

In [ ]:
D_F = rng.gamma(.16, 2500, 5000)
D_N = rng.gamma(4.1, 100, 5000)
#print(D_F)
#print(D_N)

In [ ]:
seaborn.displot(D_F)

In [ ]:
seaborn.displot(D_N)

In [ ]:
np.mean(D_F)

In [ ]:
np.mean(D_N)

In [ ]:
bF = get_n_samples_from_boostrap_estimate_of_estimator_distribution(D_F, np.mean, n=1000)
bN = get_n_samples_from_boostrap_estimate_of_estimator_distribution(D_N, np.mean, n=1000)

In [ ]:
dbF = pandas.DataFrame({r'$\hat{K}_F$': bF, 'Technologie': ['F']*len(bF)})
dbN = pandas.DataFrame({r'$\hat{K}_F$': bN, 'Technologie': ['N']*len(bN)})
b = pandas.concat([dbF, dbN])
seaborn.displot(data=b, x=r'$\hat{K}_F$', hue='Technologie', kind='hist', stat='density', common_norm=False)
plt.ylabel('Densité', fontsize=20)
plt.xlabel(r'$\hat{K}_F$', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig('boostrap_distribution_hat_KF_hat_KN_5000_1000.pdf')